# Predict the year from the abstract 

In [1]:
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import time

import string # библиотека для работы со строками
import nltk   # Natural Language Toolkit
# загружаем библиотеку для лемматизации
import pymorphy2 # Морфологический анализатор

#вычисляем tf-idf
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import Binarizer

In [2]:
from data_types import AuthorsDB
from data_types import PublicationsDB
from data_types import AbstractsDB

audb = AuthorsDB()
audb.load()
pubdb = PublicationsDB()
pubdb.load()
absdb = AbstractsDB()
absdb.load()

filename = "../data/mnid_author_dict.pkl"
with open(filename,'rb') as inp:
    authors_dict = pickle.load(inp)

In [3]:
# authors_dict2 = {v['mn_id']:v['name'] for k,v in authors_dict.items()}

In [4]:
ind = 0
for k, v in authors_dict.items():
    ind += 1
    print(k,v)
    if ind>4:
        break
        

29400 Абалакин Илья Владимирович
25875 Авдуевский Всеволод Сергеевич
79709 Агапов Владимир Михайлович
37981 Агафонов С И
39797 Агаян В М


In [5]:
# filename = '../data/mnid_author_dict.pkl'    
# with open(filename,'wb') as outp:
#     pickle.dump(authors_dict2, outp, pickle.HIGHEST_PROTOCOL)

In [6]:
# filename = "../data/mathnet_iam_authors_dict.pkl"
# with open(filename,'rb') as inp:
#     authors_dict = pickle.load(inp)

In [7]:
pub = pd.DataFrame.from_dict(pubdb.db,orient='index')

# 1 Data Preprocessing

I need to get 
- abstract and it's tokenization
- year of paper



In [8]:
abstracts = absdb.db[absdb.db['abstract'].notna()].copy()

In [9]:
absdb.db.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7597 entries, vyurv213 to vmumm4420
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   abstract  5609 non-null   object
 1   keywords  2525 non-null   object
dtypes: object(2)
memory usage: 178.1+ KB


In [10]:
abstracts.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5609 entries, vyurv213 to da31
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   abstract  5609 non-null   object
 1   keywords  2525 non-null   object
dtypes: object(2)
memory usage: 131.5+ KB


In [11]:
# загружаем список стоп-слов для русского
nltk.download('stopwords')
stop_words = nltk.corpus.stopwords.words('russian')

# примеры стоп-слов
stop_words[:10]

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\-\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со']

In [12]:
# знаки препинания
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [13]:
word_tokenizer = nltk.WordPunctTokenizer()

In [14]:
# dates = [str(x) for x in np.arange(1900, 2022)]


def process_abstractsdb(data):
    texts = []    
    
    # поочередно проходим по всем новостям в списке
    for index,row in data.iterrows():
        # print(row['abstract'])        
        # print(type(row['abstract']))
        text = row['abstract']
        if (row['keywords'] is not None):
            # print(text)
            # print(row['keywords'])
            # print(type(row['keywords']))
            text += row['keywords']                   
        text_lower = text.lower() # приводим все слова к нижнему регистру
        tokens     = word_tokenizer.tokenize(text_lower) #разбиваем екст на слова                
        tokens = [word for word in tokens if (word not in string.punctuation and word not in stop_words and not word.isnumeric())]
                      
        texts.append(tokens) # добавляем в предобработанный список
    
    return texts

In [15]:
abstracts.loc[['vyurv213','vyurv1','vyurv46']].info()

<class 'pandas.core.frame.DataFrame'>
Index: 3 entries, vyurv213 to vyurv46
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   abstract  3 non-null      object
 1   keywords  3 non-null      object
dtypes: object(2)
memory usage: 72.0+ bytes


In [16]:
# texts = process_abstractsdb(abstracts.loc[['vyurv213','vyurv1','vyurv46']])
# texts = process_abstractsdb(abstracts.loc[['vyurv213','vyurv1','vyurv46']])
texts = process_abstractsdb(abstracts)

In [17]:
texts

[['увеличивающийся',
  'рост',
  'числа',
  'компонент',
  'суперкомпьютеров',
  'приводит',
  'специалистов',
  'области',
  'hpc',
  'неблагоприятным',
  'оценкам',
  'будущих',
  'суперкомпьютеров',
  'диапазон',
  'среднего',
  'времени',
  'отказами',
  'составлять',
  'часа',
  'часов',
  'данная',
  'оценка',
  'ставит',
  'вопрос',
  'возможность',
  'проведения',
  'длительных',
  'расчетов',
  'суперкомпьютерах',
  'работе',
  'предлагается',
  'метод',
  'восстановления',
  'отказов',
  'требующий',
  'возврата',
  'большинства',
  'процессов',
  'последней',
  'контрольной',
  'точке',
  'позволить',
  'сократить',
  'накладные',
  'расходы',
  'некоторых',
  'вычислительных',
  'алгоритмов',
  'стандартный',
  'метод',
  'обеспечения',
  'отказоустойчивости',
  'заключается',
  'координированном',
  'сохранении',
  'случае',
  'отказа',
  'осуществляется',
  'возврат',
  'процессов',
  'последней',
  'контрольной',
  'точке',
  'предлагаемая',
  'стратегия',
  'заключается

In [18]:
# инициализируем лемматизатор :)
morph = pymorphy2.MorphAnalyzer()

In [19]:
for aword in texts[2]:
    aword_norm = morph.parse(aword)[0].normal_form
    print("Исходное слово: %s \tЛемматизированное: %s" % (aword, aword_norm))

Исходное слово: рассматриваются 	Лемматизированное: рассматриваться
Исходное слово: вопросы 	Лемматизированное: вопрос
Исходное слово: связанные 	Лемматизированное: связанный
Исходное слово: проведением 	Лемматизированное: проведение
Исходное слово: расчетов 	Лемматизированное: расчёт
Исходное слово: распределенных 	Лемматизированное: распределённый
Исходное слово: вычислительных 	Лемматизированное: вычислительный
Исходное слово: системах 	Лемматизированное: система
Исходное слово: компоненты 	Лемматизированное: компонент
Исходное слово: которых 	Лемматизированное: который
Исходное слово: подвержены 	Лемматизированное: подверженный
Исходное слово: отказам 	Лемматизированное: отказ
Исходное слово: работе 	Лемматизированное: работа
Исходное слово: приводятся 	Лемматизированное: приводиться
Исходное слово: определения 	Лемматизированное: определение
Исходное слово: системы 	Лемматизированное: система
Исходное слово: сбоя 	Лемматизированное: сбой
Исходное слово: ошибки 	Лемматизированное: 

In [20]:
# применяем лемматизацию ко всем текстам
print(range(len(texts)))

for i in tqdm(range(len(texts))):           # tqdm_notebook создает шкалу прогресса :)
    text_lemmatized = [morph.parse(x)[0].normal_form for x in texts[i]] # применяем лемматизацию для каждого слова в тексте
    texts[i] = ' '.join(text_lemmatized)
    # time.sleep(1)  

range(0, 5609)


  0%|          | 0/5609 [00:00<?, ?it/s]

In [21]:
# texts

https://stackoverflow.com/questions/46118910/scikit-learn-vectorizer-max-features

In [47]:
# Fit TF-IDF on train texts
# vectorizer = TfidfVectorizer(min_df=1,   max_features=None, strip_accents='unicode',  analyzer='word',token_pattern=r'\w{2,}',ngram_range=(1, 2), use_idf=1,smooth_idf=1,sublinear_tf=1)
vectorizer = TfidfVectorizer(min_df=1,   max_features=None, analyzer='word',token_pattern=r'\w{2,}', ngram_range=(1, 2), use_idf=1,smooth_idf=1,sublinear_tf=1) 
vectorizer.fit(texts)


TfidfVectorizer(ngram_range=(1, 2), smooth_idf=1, sublinear_tf=1,
                token_pattern='\\w{2,}', use_idf=1)

In [48]:
print(len(vectorizer.get_feature_names_out()))


188195


In [49]:

# Топ-10 слов
vectorizer.get_feature_names_out()[:]

array(['00458', '00458 офи_м', '00750', ..., 'ёмкость полюс',
       'ёмкость приближать', 'ёмкость сопротивление'], dtype=object)

In [50]:
tf_idf_matrix = vectorizer.fit_transform(texts)

In [51]:
print(type(tf_idf_matrix))

<class 'scipy.sparse._csr.csr_matrix'>


In [52]:
tf_idf_matrix.shape

(5609, 188195)

In [53]:
def save_vectorizer(data,status='write'):
    filename = '../data/vectorizer.pkl'    
    if status == 'write':        
        with open(filename,'wb') as outp:
            pickle.dump(data, outp, pickle.HIGHEST_PROTOCOL)
            
def save_tf_idf_matrix(data,status='write'):
    filename = '../data/tf_idf_matrix.pkl'    
    if status == 'write':        
        with open(filename,'wb') as outp:
            pickle.dump(data, outp, pickle.HIGHEST_PROTOCOL)

In [54]:
save_vectorizer(vectorizer,status="write")
save_tf_idf_matrix(tf_idf_matrix,status="write")

In [55]:
def load_vectorizer():
    filename = '../data/vectorizer.pkl'
    res = None    
    with open(filename,'rb') as inp:
        res = pickle.load(inp)
    return res

def load_tf_idf_matrix():
    filename = '../data/tf_idf_matrix.pkl'
    res = None    
    with open(filename,'rb') as inp:
        res = pickle.load(inp)
    return res


In [31]:
# x = load_vectorizer()

In [32]:
# x.get_feature_names_out()[:10]

# Importance of the article for Request

In [56]:
def process_request(data):        
    text_lower = data.lower()
    tokens = word_tokenizer.tokenize(text_lower)
    tokens = [word for word in tokens if (word not in string.punctuation and word not in stop_words and not word.isnumeric())]        
    texts = ' '.join(tokens)         
    return [morph.parse(texts)[0].normal_form]

In [57]:
from collections import defaultdict
def get_authors_rating(df, coef):
    # print(df)
    # print(type(df))
    # print(coef)
    # print(type(coef))
    rating = defaultdict(int)
    
    for index in range(len(df)):
        # print(f'index = {index}, row = {df.iloc[index]}')        
        for author in df.iloc[index]['author_id']:  
            # print(index)          
            # print(f'add coef = {coef[index]} to author = {author}')
            rating[author] += coef[index]            
            # print(rating)
    # dict(sorted(rating.items(), key=lambda item: item[1]))
    rating_list = [ (k,v) for k, v in sorted(rating.items(), key=lambda item: item[1], reverse=True)]
    return rating_list

In [35]:


# request = process_request(request)
# print(request)
# print(type(request))



In [58]:
# result =  get_model_response(request, vectorizer, tf_idf_matrix, abstracts, authors_dict)
def get_model_response(request, vectorizer, tf_idf_matrix, abstracts, authors_dict):    
    request = process_request(request)    
    vect_req = vectorizer.transform(request)
    onehot = Binarizer()
    ohe_request = onehot.fit_transform(vect_req.toarray())    
    coef= tf_idf_matrix.dot(ohe_request.T)    
    coef = coef.flatten()    
    
    max_number_of_articles = 30
    low_bound = 0.01
    max_output_pubs = 5
    max_output_authors = 5
    
    pubs_index = np.argsort(coef)[::-1][:max_number_of_articles]        
    low_bound_pub_index  = [ind for ind in pubs_index if coef[ind]>low_bound]
    req_pups = list(abstracts.iloc[low_bound_pub_index].index)    
    top_pubs = pub.loc[req_pups]['reference']
    top_autors = get_authors_rating(pub.loc[req_pups],coef[pubs_index])
        
    index = 0
    for item in top_autors:        
        if index >= max_output_authors:
            break
        if item[0] in authors_dict:
            index +=1
            print(f'author: {authors_dict[item[0]]}, raiting: {round(item[1],2)}')            
            
    for item in top_pubs[:max_output_pubs]:
        print(item)    

In [59]:
def get_model_response_print(request, vectorizer, tf_idf_matrix, abstracts, authors_dict):
    # tf_idf_matrix= train_X = vectorizer.fit_transform(texts)
    # print(request)
    # print(type(request))
    request = process_request(request)
    # print(request)
    # print(type(request))
    vect_req = vectorizer.transform(request)
    onehot = Binarizer()
    ohe_request = onehot.fit_transform(vect_req.toarray())
    # print(ohe_request.shape)
    # print(tf_idf_matrix.shape)
    coef= tf_idf_matrix.dot(ohe_request.T)
    # print(coef)
    coef = coef.flatten()
    
    
    max_number_of_articles = 30
    low_bound = 0.01
    max_output_pubs = 5
    max_output_authors = 5
    
    pubs_index = np.argsort(coef)[::-1][:max_number_of_articles]
    # print(pubs_index)
    # coef[pubs_index]
    
    low_bound_pub_index  = [ind for ind in pubs_index if coef[ind]>low_bound]
    # print(low_bound_pub_index)
    # print(coef[low_bound_pub_index])
    
    req_pups = list(abstracts.iloc[low_bound_pub_index].index)
    print(req_pups)
    print(pub.loc[req_pups].head())
    
    top_pubs = pub.loc[req_pups]['reference']
    
    top_autors = get_authors_rating(pub.loc[req_pups],coef[pubs_index])
    print(top_autors[:max_output_authors][0], top_autors[:max_output_authors][1])
    print(type(top_autors))
    print(top_autors)
    index = 0
    for item in top_autors:
        print(index, item)        
        if index >= max_output_authors:
            break
        if item[0] in authors_dict:
            index +=1
            print(f'author = {authors_dict[item[0]]}, his raiting = {round(item[1],2)}')
            
            
    for item in top_pubs[:max_output_pubs]:
        print(item)

In [60]:
request = 'суперкомпьютерное моделирование отказоустойчивые вычисления на hpc системах'
result =  get_model_response(request, vectorizer, tf_idf_matrix, abstracts, authors_dict)

author: Боровин Геннадий Константинович, raiting: 0.74
author: Четверушкин Борис Николаевич, raiting: 0.53
author: Овчинников Михаил Юрьевич, raiting: 0.42
author: Елизарова Татьяна Геннадьевна, raiting: 0.31
author: Карамзин Юрий Николаевич, raiting: 0.28
А. В. Колдоба, Ю. А. Повещенко, П. П. Матус, М. М. Чуйко, “Математическое моделирование течения жидкости в разветвленных гидравлических системах”, Матем. моделирование, 4:9 (1992), 43–54
Т. Г. Елизарова, M. Е. Соколова, Ю. В. Шеретов, “Квазигазодинамические уравнения и численное моделирований течений вязкого газа”, Ж. вычисл. матем. и матем. физ., 45:3 (2005), 545–556; Comput. Math. Math. Phys., 45:3 (2005), 524–534
Г. К. Боровин, А. В. Костюк, “Программный комплекс для моделирования динамики гидравлической системы управления шагающей машины”, Препринты ИПМ им. М. В. Келдыша, 2002, 046
Г. К. Боровин, “Некоторые особенности моделирования электромеханических систем управления”, Препринты ИПМ им. М. В. Келдыша, 1999, 072
Ю. Н. Карамзин,

In [61]:
request = 'суперкомпьютерное'
result =  get_model_response(request, vectorizer, tf_idf_matrix, abstracts, authors_dict)

author: Поляков Сергей Владимирович, raiting: 0.46
author: Горобец Андрей Владимирович, raiting: 0.42
author: Пузырьков Дмитрий Валерьевич, raiting: 0.38
author: Балута Виктор Иванович, raiting: 0.33
author: Тараканов Илья Алексеевич, raiting: 0.32
А. А. Васильев, А. Г. Жогов, О. С. Косарев, М. Б. Марков, А. И. Потапенко, Р. В. Ульяненков, И. А. Тараканов, “Экспериментальная проверка математической модели переноса фотонов и электронов”, Препринты ИПМ им. М. В. Келдыша, 2019, 084, 17 с.
С. В. Поляков, А. В. Выродов, Д. В. Пузырьков, М. В. Якобовский, “Облачный сервис для решения многомасштабных задач нанотехнологии на суперкомпьютерных системах”, Труды ИСП РАН, 27:6 (2015), 409–420
А. В. Горобец, М. И. Нейман-заде, С. К. Окунев, А. А. Калякин, С. А. Суков, “Производительность процессора Эльбрус-8С в суперкомпьютерных приложениях вычислительной газовой динамики”, Препринты ИПМ им. М. В. Келдыша, 2018, 152, 20 с.
В. П. Осипов, Б. Н. Четверушкин, В. И. Балута, Ю. И. Нечаев, “Формальный апп

In [64]:
request = 'моделирование'
result =  get_model_response(request, vectorizer, tf_idf_matrix, abstracts, authors_dict)

author: Тишкин Владимир Федорович, raiting: 0.29
author: Боровин Геннадий Константинович, raiting: 0.26
author: Повещенко Юрий Андреевич, raiting: 0.23
author: Липанов Алексей Матвеевич, raiting: 0.2
author: Самарский Александр Андреевич, raiting: 0.2
А. В. Колдоба, Ю. А. Повещенко, П. П. Матус, М. М. Чуйко, “Математическое моделирование течения жидкости в разветвленных гидравлических системах”, Матем. моделирование, 4:9 (1992), 43–54
С. А. Махов, “Пятисекторная долгосрочная макромодель мировой динамики на основе эмпирических данных”, Препринты ИПМ им. М. В. Келдыша, 2011, 072, 24 с.
Н. Г. Басов, Г. А. Вергунова, П. П. Волосевич, Е. Г. Гамалий, С. Ю. Гуськов, Г. А. Евсеев, Е. И. Леванов, В. И. Маслянкин, В. Б. Розанов, А. А. Самарский, “О преобразовании лазерного излучения в собственное тепловое излучение плазмы”, Квантовая электроника, 14:9 (1987), 1887–1893 [Quantum Electron., 17:9 (1987), 1203–1206]
Р. В. Жалнин, Н. В. Змитренко, М. Е. Ладонкина, В. Ф. Тишкин, “Численное моделирован

In [65]:
request = 'отказоустойчивые вычисления на hpc системах'
result =  get_model_response(request, vectorizer, tf_idf_matrix, abstracts, authors_dict)

author: Четверушкин Борис Николаевич, raiting: 0.45
author: Боровин Геннадий Константинович, raiting: 0.35
author: Юхно Людмила Филипповна, raiting: 0.3
author: Брюно Александр Дмитриевич, raiting: 0.27
author: Ферапонтов Евгений Владимирович, raiting: 0.27
А. А. Абрамов, В. И. Ульянова, Л. Ф. Юхно, “Об общей нелинейной самосопряженной спектральной задаче для систем обыкновенных дифференциальных уравнений”, Ж. вычисл. матем. и матем. физ., 49:4 (2009), 624–627; Comput. Math. Math. Phys., 49:4 (2009), 602–605
Ю. А. Криксин, П. Г. Халатур, А. Р. Хохлов, “Алгоритм решения интегральных уравнений для многокомпонентных полиэлектролитных систем”, Матем. моделирование, 14:7 (2002), 74–80
К. Н. Иванова, Б. Н. Четверушкин, Н. Г. Чурбанова, “Квазигазодинамическая система уравнений и уравнения Навье–Стокса”, Матем. моделирование, 16:4 (2004), 98–104
А. Д. Брюно, “Степенные асимптотики решений нелинейных систем”, Изв. АН СССР. Сер. матем., 29:2 (1965), 329–364
систем уравнений”, Дифференц. уравнени

In [66]:
request = 'отказоустойчивые вычисления'
result =  get_model_response(request, vectorizer, tf_idf_matrix, abstracts, authors_dict)

author: Юхно Людмила Филипповна, raiting: 0.54
author: Брюно Александр Дмитриевич, raiting: 0.28
author: Марченков Сергей Серафимович, raiting: 0.23
author: Малинецкий Георгий Геннадьевич, raiting: 0.23
author: Горелик Алла Моисеевна, raiting: 0.22
С. С. Марченков, В. Л. Матросов, “Сложность алгоритмов и вычислений”, Итоги науки и техн. Сер. Теор. вероятн. Мат. стат. Теор. кибернет., 16, ВИНИТИ, М., 1979, 103–149; J. Math. Sci. (N. Y.), 15:2 (1981), 140–165
А. А. Абрамов, Л. Ф. Юхно, “Один метод исключения для линейных задач”, Ж. вычисл. матем. и матем. физ., 38:4 (1998), 547–556; Comput. Math. Math. Phys., 38:4 (1998), 527–535
Г. Г. Малинецкий, С. А. Науменко, “Вычисления на ДНК. Эксперименты. Модели. Алгоритмы. Инструментальные средства”, ИТиВС, 2006, № 1, 5–27
Г. Г. Малинецкий, С. А. Науменко, “Вычисления на ДНК. Эксперименты. Модели. Алгоритмы. Инструментальные средства”, Препринты ИПМ им. М. В. Келдыша, 2005, 057
А. А. Абрамов, В. И. Ульянова, Л. Ф. Юхно, “Метод решения нелинейной

In [67]:
request = 'отказоустойчивые'
result =  get_model_response(request, vectorizer, tf_idf_matrix, abstracts, authors_dict)

author: Бондаренко Алексей Алексеевич, raiting: 0.09
author: Якобовский Михаил Владимирович, raiting: 0.09
author: Аким Эфраим Лазаревич, raiting: 0.06
author: Овчинников Михаил Юрьевич, raiting: 0.06
А. А. Бондаренко, М. В. Якобовский, “Моделирование отказов в высокопроизводительных вычислительных системах в рамках стандарта MPI и его расширения ULFM”, Вестн. ЮУрГУ. Сер. Выч. матем. информ., 4:3 (2015), 5–12
E. L. Akim, K. Bries, P. Behr, L. A. Vedeshin, D. V. Vorontsov, V. V. Egorov, S. K. Kalashnikov, A. P. Kalinin, K.-H. Kolk, S. Montenegro, M. Yu. Ovchinnikov, A. G. Orlov, S. Pletner, A. I. Rodionov, I. D. Rodionov, E. Yu. Fedunin, B. R. Shub, “The fire infrared-hyperspectral monitoring (Russian–Germany proposals for an international Earth observation mission)”, Препринты ИПМ им. М. В. Келдыша, 2004, 032, 34 pp.


In [44]:
request = 'ULFM'
result =  get_model_response(request, vectorizer, tf_idf_matrix, abstracts, authors_dict)

author: Бондаренко Алексей Алексеевич, raiting: 0.34
author: Якобовский Михаил Владимирович, raiting: 0.34
author: Ляхов Павел Александрович, raiting: 0.17
А. А. Бондаренко, М. В. Якобовский, “Моделирование отказов в высокопроизводительных вычислительных системах в рамках стандарта MPI и его расширения ULFM”, Вестн. ЮУрГУ. Сер. Выч. матем. информ., 4:3 (2015), 5–12
А. А. Бондаренко, П. А. Ляхов, М. В. Якобовский, “Координированное сохранение с журналированием передаваемых данных и асинхронное восстановление в случае отказа”, Вестн. ЮУрГУ. Сер. Выч. матем. информ., 8:2 (2019), 76–91


In [68]:
request = 'вычисления'
result =  get_model_response(request, vectorizer, tf_idf_matrix, abstracts, authors_dict)

author: Юхно Людмила Филипповна, raiting: 0.54
author: Брюно Александр Дмитриевич, raiting: 0.28
author: Марченков Сергей Серафимович, raiting: 0.23
author: Малинецкий Георгий Геннадьевич, raiting: 0.23
author: Горелик Алла Моисеевна, raiting: 0.22
С. С. Марченков, В. Л. Матросов, “Сложность алгоритмов и вычислений”, Итоги науки и техн. Сер. Теор. вероятн. Мат. стат. Теор. кибернет., 16, ВИНИТИ, М., 1979, 103–149; J. Math. Sci. (N. Y.), 15:2 (1981), 140–165
А. А. Абрамов, Л. Ф. Юхно, “Один метод исключения для линейных задач”, Ж. вычисл. матем. и матем. физ., 38:4 (1998), 547–556; Comput. Math. Math. Phys., 38:4 (1998), 527–535
Г. Г. Малинецкий, С. А. Науменко, “Вычисления на ДНК. Эксперименты. Модели. Алгоритмы. Инструментальные средства”, ИТиВС, 2006, № 1, 5–27
Г. Г. Малинецкий, С. А. Науменко, “Вычисления на ДНК. Эксперименты. Модели. Алгоритмы. Инструментальные средства”, Препринты ИПМ им. М. В. Келдыша, 2005, 057
А. А. Абрамов, В. И. Ульянова, Л. Ф. Юхно, “Метод решения нелинейной

In [69]:
request = 'интеллект'
result =  get_model_response(request, vectorizer, tf_idf_matrix, abstracts, authors_dict)

author: Платонов Александр Константинович, raiting: 0.43
author: Малинецкий Георгий Геннадьевич, raiting: 0.28
author: Ахтёров А В, raiting: 0.15
author: Орлов Юрий Николаевич, raiting: 0.15
author: Фролов Владимир Александрович, raiting: 0.1
О. И. Давыдов, А. К. Платонов, “Робот и Искусственный Интеллект. Технократический подход”, Препринты ИПМ им. М. В. Келдыша, 2017, 112, 24 с.
А. А. Кирильченко, А. К. Платонов, В. Е. Пряничников, Ю. И. Степанов, О. Н. Трубицин, “Экспертная поддержка систем управления мобильными роботами”, Препринты ИПМ им. М. В. Келдыша, 1996, 057
А. А. Зацаринный, Н. И. Ильин, К. К. Колин, В. Е. Лепский, Г. Г. Малинецкий, Д. А. Новиков, А. Н. Райков, С. Н. Сильвестров, Б. Б. Славин, “Ситуационные центры развития в полисубъектной среде”, Пробл. управл., 2017, № 5, 31–42
В. В. Санжаров, В. А. Фролов, А. Г. Волобой, В. А. Галактионов, Д. С. Павлов, “Система генерации наборов изображений для задач компьютерного зрения на основе фотореалистичного рендеринга”, Препринты

In [70]:
request = 'искусственный'
result =  get_model_response(request, vectorizer, tf_idf_matrix, abstracts, authors_dict)

author: Попов Игорь Викторович, raiting: 1.96
author: Фрязинов Игорь Владимирович, raiting: 1.0
author: Мишков М Н, raiting: 0.31
author: Повещенко Юрий Андреевич, raiting: 0.29
author: Ахтёров А В, raiting: 0.27
И. В. Попов, Ю. Е. Тимофеева, “Построение разностной схемы повышенного порядка аппроксимации для уравнения переноса с использованием адаптивной искусственной вязкости”, Препринты ИПМ им. М. В. Келдыша, 2015, 039, 25 с.
И. В. Попов, И. В. Фрязинов, “О новом выборе адаптивной искусственной вязкости”, Матем. моделирование, 22:12 (2010), 23–32; Math. Models Comput. Simul., 3:4 (2011), 411–418
И. В. Попов, “Метод адаптивной искусственной вязкости для численного решения гиперболических уравнений и систем”, Препринты ИПМ им. М. В. Келдыша, 2020, 034, 18 с.
И. В. Попов, И. В. Фрязинов, “Метод адаптивной искусственной вязкости для уравнений газовой динамики на треугольных и тетраэдральных сетках”, Матем. моделирование, 24:6 (2012), 109–127; Math. Models Comput. Simul., 5:1 (2013), 50–6

In [71]:
request = 'искусственный интеллект'
result =  get_model_response(request, vectorizer, tf_idf_matrix, abstracts, authors_dict)

author: Попов Игорь Викторович, raiting: 1.74
author: Платонов Александр Константинович, raiting: 1.0
author: Фрязинов Игорь Владимирович, raiting: 0.78
author: Малинецкий Георгий Геннадьевич, raiting: 0.78
author: Ахтёров А В, raiting: 0.58
О. И. Давыдов, А. К. Платонов, “Робот и Искусственный Интеллект. Технократический подход”, Препринты ИПМ им. М. В. Келдыша, 2017, 112, 24 с.
А. А. Кирильченко, А. К. Платонов, В. Е. Пряничников, Ю. И. Степанов, О. Н. Трубицин, “Экспертная поддержка систем управления мобильными роботами”, Препринты ИПМ им. М. В. Келдыша, 1996, 057
А. А. Зацаринный, Н. И. Ильин, К. К. Колин, В. Е. Лепский, Г. Г. Малинецкий, Д. А. Новиков, А. Н. Райков, С. Н. Сильвестров, Б. Б. Славин, “Ситуационные центры развития в полисубъектной среде”, Пробл. управл., 2017, № 5, 31–42
Ю. Ф. Голубев, “Нейронные сети в мехатронике”, Фундамент. и прикл. матем., 11:8 (2005), 81–103; J. Math. Sci., 147:2 (2007), 6607–6622
А. В. Ахтёров, А. А. Кирильченко, “Основы теоретической робототе